# Neighborhoods with Plant-based Restaurants in Toronto

In [1]:
!pip install bs4

     |████████████████████████████████| 122kB 34.9MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import numpy as np 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         235 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2021.4.13-~ --> conda-forge::ca-certificates-

## Get Toronto Neighborhood Data

### Retrieve the Toronto neighborhood data at Wikipedia page and create a Beautiful soup object

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [ ]:
data  = requests.get(url).text 

In [ ]:
soup = BeautifulSoup(data,"html5lib")

### Scrape and clean the data

In [ ]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print(table_contents)

### Create a dataframe with the list of neighborhoods

In [ ]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [ ]:
df.head()

In [ ]:
df.describe()

## Get Latitude and Longitude Coordinates of Toronto neighborhoods

### Get geographical coordinates of each neighborhood from a csv file

In [ ]:
url_csv = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
ll_df = pd.read_csv(url_csv)
#ll_df.head()

In [ ]:
ll_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [ ]:
ll_df.head()

### Create a new dataframe containing postal code, borought, neighborhood, latitude and longitude information

In [ ]:
neighborhoods = pd.merge(df, ll_df, on='PostalCode')

In [ ]:
neighborhoods

## Map of the neighborhoods in Toronto

### Use geopy library to get the latitude and longitude values of Toronto

In [ ]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

### Create a map of Toronto with neighborhoods superimposed on top.

In [ ]:
map_toronto_all = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_all)  
    
map_toronto_all

## Explore each neighborhood

### Define Foursquare credentials to use the Foursquare API to explore the neighborhoods and segment them.

In [ ]:
CLIENT_ID = 'WZ0XRMER3OZZEEDUZKRATHBV3NLZAWBT2XMYNNNOZTV3T0UN' 
CLIENT_SECRET = 'LCQQR0KXS2V3CNNCNB0KITDIBLTFEZOBDMWPD0MWUEN3ILIX' 
VERSION = '20180604'
LIMIT = 100

### Create function to get nearby venues for each neighborhodd

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Run the above function on each neighborhood and create a new dataframe

In [ ]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

In [ ]:
toronto_venues.head()

## Analyze each neighborhood

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# rename "Neighborhood" venue category to "Neighborhood in General"
toronto_onehot.rename(columns={'Neighborhood':'Neighborhood in General'},inplace=True)

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

### Create new dataframe with "vegetarian/vegan restaurants" only

In [ ]:
toronto_grouped_veg = toronto_grouped[["Neighborhood","Vegetarian / Vegan Restaurant"]]

In [ ]:
toronto_grouped_veg.head()

## Cluster neighborhoods

### Run k-means to cluster the neighborhood into 3 clusters

In [ ]:
kclusters = 3

toronto_grouped_veg_clustering = toronto_grouped_veg.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_veg_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

### Create a new dataframe that includes cluster labels and geographical data

In [ ]:
# create new dataframe with cluster labels
toronto_merged = toronto_grouped_veg

toronto_merged.insert(0, 'Cluster Labels', kmeans.labels_)

# merge with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

### Sort results by frequency of vegetarian/vegan restaurants

In [ ]:
toronto_merged.sort_values(['Vegetarian / Vegan Restaurant'], ascending=False, inplace=True)
toronto_merged

### Map of the clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine clusters

### Cluster 0

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

### Cluster 1

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

### Cluster 2

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]